# Spam & Ham

In [113]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from tensorflow.keras.preprocessing.text import Tokenizer

In [98]:
data = pd.read_csv('./data/spam_datasets.csv', encoding='latin1')
data.dropna(axis=1, inplace=True)
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [99]:
data.v2 = data.v2.str.replace("[^\w]|br", " ")

data.v2 = data.v2.replace("", np.nan)
data.dropna(how='any', inplace=True)

In [100]:
data.head()

,v1,v2
0,ham,Go until jurong point crazy Available only ...
1,ham,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor U c already then say
4,ham,Nah I don t think he goes to usf he lives aro...


In [103]:
# ham = 1, spam = 0
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])

In [104]:
data.head()

,v1,v2
0,0,Go until jurong point crazy Available only ...
1,0,Ok lar Joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor U c already then say
4,0,Nah I don t think he goes to usf he lives aro...


In [105]:
text_train, text_test, y_train, y_test = train_test_split(data['v2'], data['v1'], random_state=123)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4179,), (1393,), (4179,), (1393,))

In [ ]:
# 불용어
stop_words = set(stopwords.words('english')) 

In [110]:
X_train = []

for words in text_train:
    for w in word_tokenize(words):
        if w not in stop_words:
            X_train.append(w)

# print(X_train)

In [111]:
X_test = []

for words in text_test:
    for w in word_tokenize(words):
        if w not in stop_words:
            X_test.append(w)

In [114]:
# X_train 단어들을 토대로 정수 인덱스 설정, 전체 단어 갯수 설정
tokenizer = Tokenizer(20000)
tokenizer.fit_on_texts(X_train)

# 설정된 정수 인덱스를 토대로 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print('# int_encoding done')

# int_encoding done


밑에는 추가 공부 필요함!

In [115]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [116]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

# 레이어들을 쌓을 모델을 생성
model = Sequential()
# 단어를 임베딩하는데, 5000개 (imdb) 혹은 20000개 (네이버) 의 단어를 120차원으로 내보내겠다
# 1인자 = 내가 넣을 단어의 갯수 (총 인덱스의 갯수), 2인자 = 출력할 차원 (덴스 벡터의 차원), 3인자 = 문장의 길이
model.add(Embedding(5000, 120))
# RNN - simpleRNN / LSTM
model.add(LSTM(120))
# 긍정/부정을 판단하니까 이진 분류 -> sigmoid 함수 사용
model.add(Dense(1, activation='sigmoid'))

# 혹시 5회 이상 검증데이터 loss가 증가하면, 과적합될 수 있으므로 학습을 조기종료!
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# 훈련을 거듭하면서, 가장 검증데이터 정확도가 높았던 순간을 체크포인트로 저장
model_check = ModelCheckpoint('the_best.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

# 긍정/부정을 판단하니까 손실함수는 이진 교차 엔트로피, 최적화는 adam, 평가 기준은 acc (출력할때 뜬다)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=64, callbacks=[early_stop, model_check])

# 정확도 측정
# 출력하면 [loss, acc]
print(model.evaluate(X_test, y_test))
# X 값은 전처리, 토큰화, 정수인코딩이 된 상태의 문장이어야
print(model.predict(X))

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, <class 'pandas.core.series.Series'>